In [1]:
import pandas as pd
import numpy as np
import re
import ast
from ast import literal_eval
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
%matplotlib notebook

In [2]:
file = '../data/spain-red'
df = pd.read_csv(file + '.csv', converters={'foods': literal_eval, 'highlights': literal_eval})

In [3]:
diacritics = {'&#237;': 'í', '&#243;': 'ó', '&#250;': 'ú', '&#241;': 'ñ', '&#232;': 'è', \
    '&#8364;': '€', '&#193;': 'Á', '&#192;': 'À', '&#233;': 'é', '&#224;': 'à', \
    '&#239;': 'ï', '&#231;': 'ç', '&#193;': 'Á', '&#252;': 'ü', '&#225;': 'á', '&#186;': 'º', \
    '&#244;': 'ô', '&#8217;': '’', '&#960;': 'π', '&#210;': 'Ò'}
df.replace(diacritics, regex=True, inplace=True)
df['winery'] = df['winery'].apply(lambda x: re.sub(r'((?<=[a-z_à-ÿ])[A-Z]|(?<!\A)[A-Z](?=[a-z_à-ÿ]))', r' \1', x))
df['vintage'] = df['vintage'].apply(lambda x: re.sub(r'((?<=[a-z_à-ÿ])[A-Z]|(?<!\A)[A-Z](?=[a-z_à-ÿ]))', r' \1', x))
df['region'] = df['region'].apply(lambda x: re.sub(r'((?<=[a-z_à-ÿ])[A-Z]|(?<!\A)[A-Z](?=[a-z_à-ÿ]))', r' \1', x))
df['vintage'] = df['vintage'].apply(lambda x: re.sub(r"(\S)\(", r'\1 (', x))
df['vintage'] = df['vintage'].apply(lambda x: re.sub('\(\s*(.*?)\s*\)', r'(\1)', x))
df['vintage'] = df['vintage'].apply(lambda x: re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", x).strip())
df['vintage'] = df['vintage'].replace(r'\s+', ' ', regex=True)

numeric_columns = ['rating', 'ratings_count', 'country_rank', 'region_rank', \
                   'winery_rank', 'global_rank', 'alcohol', 'body', 'acidity', 'price']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

df = df.drop(['drink_from', 'drink_until'], axis = 1)

for f in df['foods']:
    f.sort()

In [4]:
replace_dict = {'2 times': '100%', '3 times': '200%', '4 times': '300%', \
                '5 times': '400%', 'as much.': 'more.'}

for key, value in replace_dict.items():
    df['highlights'] = df['highlights'].astype(str).str.replace(key, value).apply(ast.literal_eval)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_5664\1231605690.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['highlights'] = df['highlights'].astype(str).str.replace(key, value).apply(ast.literal_eval)


In [5]:
highlights = df[(df['highlights'] != '[]')]['highlights']

h_list = []

for highlight in highlights:
    if highlight:
        for h in highlight:
            h_list.append(h)
            
unique = list(dict.fromkeys(h_list))
print(unique)

["Featured in Vivino's 2020 Wine Style Awards: Spanish Ribera Del Duero Red", 'Oldest vintage available', '98 by Gu&#237;a Vivir el Vino', 'Latest vintage available', 'Great value for money. Similar wines usually cost 100% more.', 'Cheapest vintage available', 'This vintage rates better than any other year for this wine', 'Among top 1% of all wines in the world', "Featured in Vivino's 2020 Wine Style Awards: Spanish Syrah", '5 by Mundus Vini', 'Popular among Vivino users. More than 200 ratings', "Featured in Vivino's 2020 Wine Style Awards: Spanish Cabernet Sauvignon", 'Popular among Vivino users. More than 500 ratings', '93 by Wine Spectator', 'Popular among Vivino users. More than 7200 ratings', 'Good value for money. Similar wines usually cost 50% more.', 'Popular among Vivino users. More than 80 ratings', 'Popular among Vivino users. More than 20 ratings', 'Among top 4% of all wines in the world', "Featured in Vivino's 2018 Wine Style Awards: Spanish Montsant Red", 'Popular among V

In [6]:
good_value_hl = []

for highlight in highlights:
    if highlight:
        for h in highlight:
            m = re.search('Similar wines usually cost(.+?)more.', h)
            if m:
                good_value_hl.append(highlight)

In [7]:
good_value_hl

[['Great value for money. Similar wines usually cost 100% more.',
  'Cheapest vintage available'],
 ['Good value for money. Similar wines usually cost 50% more.',
  'Among top 1% of all wines in the world',
  'Latest vintage available'],
 ['Great value for money. Similar wines usually cost 400% more.',
  'This vintage rates better than any other year for this wine',
  'Among top 3% of all wines in the world'],
 ['Good value for money. Similar wines usually cost 74% more.',
  '94 by Tim Atkin'],
 ['Great value for money. Similar wines usually cost 200% more.',
  'This vintage rates better than any other year for this wine',
  'Oldest vintage available'],
 ['Good value for money. Similar wines usually cost 51% more.'],
 ['Good value for money. Similar wines usually cost 43% more.',
  'Among top 5% of all wines in the world'],
 ['Great value for money. Similar wines usually cost 100% more.',
  'This vintage rates better than any other year for this wine',
  'Among top 2% of all wines in t

In [8]:
df_gv = df[df['highlights'].isin(good_value_hl)]
df_gv.describe()

,rating,ratings_count,country_rank,region_rank,winery_rank,global_rank,alcohol,body,acidity,price
count,293.000000,293.000000,293.000000,293.000000,293.000000,293.000000,230.000000,280.000000,280.000000,290.000000
mean,4.129693,720.621160,4.095563,5.283276,10.754266,4.334471,14.078261,4.257143,2.967857,23.689862
std,0.220147,2230.164805,3.648665,4.988853,11.000048,3.559293,1.725295,0.506183,0.176695,21.334646
min,3.800000,26.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,2.000000,4.900000
25%,3.900000,84.000000,1.000000,2.000000,3.000000,2.000000,14.000000,4.000000,3.000000,10.950000
50%,4.100000,206.000000,3.000000,4.000000,7.000000,4.000000,14.000000,4.000000,3.000000,15.725000
75%,4.300000,614.000000,5.000000,7.000000,14.000000,6.000000,14.500000,5.000000,3.000000,27.150000
max,4.800000,32378.000000,21.000000,27.000000,63.000000,20.000000,16.000000,5.000000,3.000000,129.000000


In [9]:
good_value = []

for lists in df_gv['highlights']:
    for l in lists:
        m = re.search('Similar wines usually cost(.+?)more.', l)
        if m:
            good_value.append(m.group(1))

df_gv['good_value'] = good_value
df_gv.head()

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_5664\75877815.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gv['good_value'] = good_value


,winery,vintage,year,region,rating,ratings_count,country_rank,region_rank,winery_rank,global_rank,...,alcohol,style,body,body_desc,acidity,acidity_desc,grapes,price,highlights,good_value
2,Aalto,PS (Pagos Seleccionados) Riberadel Duero,2018,Riberadel Duero,4.6,3231,1,1,4,1,...,15.0,Spanish Ribera Del Duero Red,5.0,Very full-bodied,3.0,High,Tempranillo,84.90,[Great value for money. Similar wines usually ...,100%
17,Abadía Retuerta,Selección Especial,2017,Sardónde Duero,4.4,3026,1,1,2,1,...,14.5,Spanish Red,4.0,Full-bodied,3.0,High,Shiraz/Syrah,26.33,[Good value for money. Similar wines usually c...,50%
29,Agustín Cubero,Stylo Garnacha Old Vines,2020,Calatayud,4.1,144,3,1,1,3,...,NaN,Spanish Grenache,5.0,Very full-bodied,3.0,High,Garnacha,8.20,[Great value for money. Similar wines usually ...,400%
43,Alión,Riberadel Duero,2017,Riberadel Duero,4.6,2179,1,1,3,1,...,15.0,Spanish Ribera Del Duero Red,5.0,Very full-bodied,3.0,High,Tempranillo,94.50,[Good value for money. Similar wines usually c...,74%
53,Altavins,Jorn Nou Petit Tinto,2020,Terra Alta,4.2,37,6,4,10,7,...,NaN,Spanish Red,NaN,NaN,NaN,NaN,NaN,9.20,[Great value for money. Similar wines usually ...,200%


In [10]:
good_value_wines = df_gv[['winery', 'vintage', 'year', 'price', 'good_value']]
good_value_wines['good_value'] = [int(row.strip("%\n ")) for row in good_value_wines['good_value']]
good_value_wines = good_value_wines[(good_value_wines['price'].notna()) & (good_value_wines['good_value'] >= 100)]
good_value_wines = good_value_wines.sort_values(by='good_value', ascending=False)
good_value_wines

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_5664\3387788540.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_value_wines['good_value'] = [int(row.strip("%\n ")) for row in good_value_wines['good_value']]


,winery,vintage,year,price,good_value
29,Agustín Cubero,Stylo Garnacha Old Vines,2020,8.20,400
868,De Muller,Legítim Crianza,2017,9.72,400
157,Artadi,Valdegines,2019,33.67,300
416,Carmelo Rodero,Pago De Valtarreña,2015,49.50,300
891,Inurrieta,Mima'o Garnacha,2020,12.90,200
...,...,...,...,...,...
917,Portaldel Priorat,Closdel Portal Negrede Negres,2018,25.92,100
920,Portaldel Priorat,Closdel Portal Somni,2016,39.51,100
1006,Fincadelos Arandinos,Crianza,2016,9.90,100
1049,Gratavinum,2 πR Priorat,2017,25.50,100


In [11]:
good_value_wines.to_html('../data/great_value_wines.html', encoding='utf-8')